In [3]:
!pip install fredapi > null

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

from fredapi import Fred

fred_key = '9ca944cf88a4cf16b87a3d012ec8bf29'

In [8]:
#FRED object
fred = Fred(api_key=fred_key)

In [14]:
sp_search = fred.search('S&P', order_by='popularity')

In [15]:
#Pull Raw Data
fred.get_series('SP500')

2014-07-16    1981.57
2014-07-17    1958.12
2014-07-18    1978.22
2014-07-21    1973.63
2014-07-22    1983.53
               ...   
2024-07-09    5576.98
2024-07-10    5633.91
2024-07-11    5584.54
2024-07-12    5615.35
2024-07-15    5631.22
Length: 2609, dtype: float64